In [1]:
# train.py
#!/usr/bin/env	python3

""" train network using pytorch

author baiyu
"""

import os
import sys
import argparse
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
#from dataset import *
from torch.autograd import Variable

from tensorboardX import SummaryWriter

from conf import settings
from utils import get_network, get_training_dataloader, get_test_dataloader, WarmUpLR

def train(epoch):
    train_loss = 0.0
    net.train()
    for batch_index, (images, labels) in enumerate(cifar100_training_loader):
        if epoch <= args.warm:
            warmup_scheduler.step()

        images = Variable(images)
        labels = Variable(labels)

        labels = labels.cuda()
        images = images.cuda()

        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        n_iter = (epoch - 1) * len(cifar100_training_loader) + batch_index + 1

#         last_layer = list(net.children())[-1]
#         for name, para in last_layer.named_parameters():
#             if 'weight' in name:
#                 writer.add_scalar('LastLayerGradients/grad_norm2_weights', para.grad.norm(), n_iter)
#             if 'bias' in name:
#                 writer.add_scalar('LastLayerGradients/grad_norm2_bias', para.grad.norm(), n_iter)

    print('Training Epoch: {epoch} \tLoss: {:0.4f}\tLR: {:0.6f}'.format(
        train_loss/len(cifar100_training_loader.dataset),
        optimizer.param_groups[0]['lr'],
        epoch=epoch,
    ))
#     update training loss for each iteration
    writer.add_scalar('Train/loss', train_loss/len(cifar100_training_loader.dataset), epoch)

    for name, param in net.named_parameters():
        layer, attr = os.path.splitext(name)
        attr = attr[1:]
        writer.add_histogram("{}/{}".format(layer, attr), param, epoch)
        
def eval_training(epoch):
    net.eval()

    test_loss = 0.0 # cost function error
    correct = 0.0

    for (images, labels) in cifar100_test_loader:
        images = Variable(images)
        labels = Variable(labels)

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        loss = loss_function(outputs, labels)
        test_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum()

    print('Test set: Average loss: {:.4f}, Accuracy: {:.4f}'.format(
        test_loss / len(cifar100_test_loader.dataset),
        correct.float()*100 / len(cifar100_test_loader.dataset)
    ))
    print()

    #add informations to tensorboard
    writer.add_scalar('Test/Average loss', test_loss / len(cifar100_test_loader.dataset), epoch)
    writer.add_scalar('Test/Accuracy', correct.float()*100 / len(cifar100_test_loader.dataset), epoch)

    return correct.float()*100 / len(cifar100_test_loader.dataset)



In [2]:
class args():
    net = 'resnet18'
    gpu = True
    w = 2
    b = 128
    s = True
    warm = 1
    lr = 0.1

In [3]:
import torchvision

In [ ]:


# net = get_network(args, use_gpu=args.gpu)
net = torchvision.models.resnet18(num_classes = 100).cuda()

#data preprocessing:
cifar100_training_loader = get_training_dataloader(
    settings.CIFAR100_TRAIN_MEAN,
    settings.CIFAR100_TRAIN_STD,
    num_workers=args.w,
    batch_size=args.b,
    shuffle=args.s
)

cifar100_test_loader = get_test_dataloader(
    settings.CIFAR100_TRAIN_MEAN,
    settings.CIFAR100_TRAIN_STD,
    num_workers=args.w,
    batch_size=args.b,
    shuffle=args.s
)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=settings.MILESTONES, gamma=0.2) #learning rate decay
iter_per_epoch = len(cifar100_training_loader)
warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * args.warm)
checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, args.net, settings.TIME_NOW)

#use tensorboard
if not os.path.exists(settings.LOG_DIR):
    os.mkdir(settings.LOG_DIR)
    
log_dir = 'test_resnet18'
writer = SummaryWriter(log_dir="./log/"+log_dir)
# input_tensor = torch.Tensor(12, 3, 32, 32).cuda()
# writer.add_graph(net, Variable(input_tensor, requires_grad=True))

#create checkpoint folder to save model
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{net}-{epoch}-{type}.pth')

best_acc = 0.0
for epoch in range(1, settings.EPOCH):
    if epoch > args.warm:
        train_scheduler.step(epoch)

    train(epoch)
    acc = eval_training(epoch)

#     #start to save best performance model after learning rate decay to 0.01 
#     if epoch > settings.MILESTONES[1] and best_acc < acc:
#         torch.save(net.state_dict(), checkpoint_path.format(net=args.net, epoch=epoch, type='best'))
#         best_acc = acc
#         continue

#     if not epoch % settings.SAVE_EPOCH:
#         torch.save(net.state_dict(), checkpoint_path.format(net=args.net, epoch=epoch, type='regular'))

# writer.close()


Files already downloaded and verified
Files already downloaded and verified


/home/weihao_zhuang/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Training Epoch: 1 	Loss: 0.0318	LR: 0.100000
Test set: Average loss: 0.0289, Accuracy: 0.1355

Training Epoch: 2 	Loss: 0.0271	LR: 0.000800
Test set: Average loss: 0.0264, Accuracy: 0.1870

